<a href="https://colab.research.google.com/github/mati-avila/FIA-TP5/blob/main/FIA2023_TPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ALUMNO: AVILA, JORGE MATIAS - LU1738 - LICENCIATURA EN SISTEMAS

In [ ]:
install.packages('lubridate')#para trabajar con fechas
install.packages('ggplot2')#Para graficos
install.packages('ggdist')#Para graficos (histogramas)

In [ ]:
install.packages("e1071")#svm
install.packages("randomForest") #randomForest
install.packages("clock") #necesaria para instalar 'caret'
install.packages("fastDummies") #para realizar el casteo (one-hot encoding)

In [ ]:
install.packages("caret") #para el particionado del df

In [ ]:
library(lubridate)
library(ggplot2)
library(ggdist)
library(dplyr)

In [5]:
# Los .csv deben estar cargados en el entorno, click en los tres puntos y copiamos la ruta de acceso
df_princess <- read.csv("/content/princess.csv")
df_wine <- read.csv("/content/wine.csv")

In [ ]:
head(df_princess) # muestra las primeras filas del dataset, si queremos ver las últimas filas usamos tail()

In [ ]:
head(df_wine) # muestra las primeras filas del dataset, si queremos ver las últimas filas usamos tail()

**Seleccion**

Seleccione el conjunto de datos sobre vinos debido a su relevancia e interés para el análisis. Podemos decir que este conjunto consta de 1599 observaciones distribuidas en 13 variables y ofrece una variedad de aspectos de los vinos, lo que permite explorar relaciones y patrones interesantes.
La calidad del vino la hemos clasificado en dos categorías: 'quality_bad' (calidad mala) y 'quality_good' (calidad buena), proporcionando una dimensión adicional para el análisis.

# LIMPIEZA

In [ ]:
# Vemos si hay campos nulos en alguna columna
colSums(is.na(df_wine))

Como el data set esta limpio, y esta baleanceado. Por lo tanto se comienza a entrenar.

Pero una propuesta para baleancear los datos, seria duplicar aleatoriamente instancias de la clase en el conjunto de datos. Esto se logra mediante la replicación de las filas correspondientes a esa clase.

In [ ]:
#importamos la libreria
library(e1071)
library(randomForest)
library(dplyr)
library(caret)
library(reshape2)
library(fastDummies)

# SVM

In [7]:
columnas_a_castear <- c('quality')
# Utilizamos fastDummies para convertir las columnas en filas
df_wineSVM <- fastDummies::dummy_cols(df_wine, select_columns = columnas_a_castear)

In [ ]:
str(df_wineSVM)

In [10]:
df_wineSVM$quality_good <- factor(df_wineSVM$quality_good)

In [11]:
# Eliminamos las columnas en df_credit_cardSVM
df_wineSVM <- df_wineSVM[, !(names(df_wineSVM) %in% columnas_a_castear)]

In [ ]:
table(df_wineSVM$quality_good)

In [13]:
set.seed(2023) #seteamos la semilla para que el experimento sea reproducible
t.ids <- createDataPartition(df_wineSVM$quality_good, p =0.7, list=F) #particionamos el df en 70% para entrenamiento y 30% para prueba
modelo <- svm(quality_good ~ ., data = df_wineSVM[t.ids, ]) #entrenamiento
prediccion <- predict(modelo, df_wineSVM[-t.ids, ]) #prueba
matriz_confusion <- table(df_wineSVM$quality_good[-t.ids], prediccion, dnn = c('Real', 'Predicho')) #creamos una matriz de confusión

In [ ]:
print(matriz_confusion)

In [ ]:
sum(prediccion != df_wineSVM$quality_good[-t.ids])/length(df_wineSVM$quality_good[-t.ids])

In [ ]:
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion)

In [ ]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

Hacemos 100 Pasadas

In [16]:
# Inicializamos vectores para almacenar los resultados
precision_vector <- numeric(100)
sensibilidad_vector <- numeric(100)

# Establecemos la semilla para reproducibilidad
set.seed(2023)

# Realizamos 100 pasadas
for (i in 1:100) {
  # Particionamos los datos
  t.ids <- createDataPartition(df_wineSVM$quality_good, p = 0.7, list = FALSE)

  # Entrenamos el modelo
  modelo <- svm(quality_good ~ ., data = df_wineSVM[t.ids, ])

  # Realizamos la predicción
  prediccion <- predict(modelo, df_wineSVM[-t.ids, ])

  # Creamos una matriz de confusión
  matriz_confusion <- table(df_wineSVM$quality_good[-t.ids], prediccion, dnn = c('Real', 'Predicho'))

  # Calculamos la precisión y sensibilidad
  precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)
  sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

  # Almacenamos los resultados en los vectores
  precision_vector[i] <- precision
  sensibilidad_vector[i] <- sensibilidad
}

# Calculamos el promedio de la precisión y sensibilidad
promedio_precision <- mean(precision_vector)
promedio_sensibilidad <- mean(sensibilidad_vector)

# Imprimimos los resultados
print(paste("Promedio de Precisión:", promedio_precision))
print(paste("Promedio de Sensibilidad:", promedio_sensibilidad))


[1] "Promedio de Precisión: 0.994634655532359"
[1] "Promedio de Sensibilidad: 0.994765625"


# RANDONFOREST

In [74]:
set.seed(2023)
t.ids <- createDataPartition(df_wineSVM$quality_good, p =0.7, list=F)
modelo_rf <- randomForest(quality_good ~ ., data = df_wineSVM[t.ids, ])
prediccion_rf <- predict(modelo_rf, newdata = df_wineSVM[-t.ids, ])
matriz_confusion <- table(prediccion_rf, df_wineSVM$quality_good[-t.ids], dnn = c('Real' ,'Predicho'))

In [ ]:
print(matriz_confusion)

In [ ]:
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion)

In [ ]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

Hacemos 100 pasadas

In [17]:
# Inicializamos vectores para almacenar los resultados
precision_vector_rf <- numeric(100)
sensibilidad_vector_rf <- numeric(100)

# Establecemos la semilla para reproducibilidad
set.seed(2023)

# Realizamos 100 pasadas
for (i in 1:100) {
  # Particionamos los datos
  t.ids <- createDataPartition(df_wineSVM$quality_good, p = 0.7, list = FALSE)

  # Entrenamos el modelo de Random Forest
  modelo_rf <- randomForest(quality_good ~ ., data = df_wineSVM[t.ids, ])

  # Realizamos la predicción con Random Forest
  prediccion_rf <- predict(modelo_rf, newdata = df_wineSVM[-t.ids, ])

  # Creamos una matriz de confusión
  matriz_confusion_rf <- table(prediccion_rf, df_wineSVM$quality_good[-t.ids], dnn = c('Real', 'Predicho'))

  # Calculamos la precisión y sensibilidad con Random Forest
  precision_rf <- (matriz_confusion_rf[1, 1] + matriz_confusion_rf[2, 2]) / sum(matriz_confusion_rf)
  sensibilidad_rf <- matriz_confusion_rf[2, 2] / sum(matriz_confusion_rf[2, ])

  # Almacenamos los resultados en los vectores
  precision_vector_rf[i] <- precision_rf
  sensibilidad_vector_rf[i] <- sensibilidad_rf
}

# Calculamos el promedio de la precisión y sensibilidad con Random Forest
promedio_precision_rf <- mean(precision_vector_rf)
promedio_sensibilidad_rf <- mean(sensibilidad_vector_rf)

# Imprimimos los resultados con Random Forest
print(paste("Promedio de Precisión con Random Forest:", promedio_precision_rf))
print(paste("Promedio de Sensibilidad con Random Forest:", promedio_sensibilidad_rf))


[1] "Promedio de Precisión con Random Forest: 1"
[1] "Promedio de Sensibilidad con Random Forest: 1"


# Redes Neuronales

In [ ]:
install.packages('neuralnet')#para trabajar con redes neuronales

In [ ]:
# Instala y carga el paquete neuralnet si aún no lo has hecho
# install.packages("neuralnet")
library(neuralnet)

In [30]:
# Configuración de la semilla para reproducibilidad
set.seed(2023)

# Partición de datos
t.ids <- createDataPartition(df_wineSVM$quality_good, p = 0.7, list = FALSE)

# Creación del objeto formula
formula <- as.formula("quality_good ~ .")

# Creación del modelo de red neuronal
modelo <- neuralnet(
  formula,
  data = df_wineSVM[t.ids, ],
  hidden = c(5, 2), # Número de neuronas en las capas ocultas
  linear.output = TRUE # Utiliza salida lineal
)

# Predicciones
prediccion <- predict(modelo, newdata = df_wineSVM[-t.ids, ])
prediccion_binaria <- ifelse(prediccion[, 1] > 1, 0, 1)

# Matriz de confusión
matriz_confusion <- table(
  Real = df_wineSVM$quality_good[-t.ids],
  Predicho = prediccion_binaria
)



In [ ]:
print(matriz_confusion)

In [ ]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

Hacemos 100 pasadas

In [33]:
# Inicializamos vectores para almacenar los resultados
precision_vector_nn <- numeric(100)
sensibilidad_vector_nn <- numeric(100)

# Configuración de la semilla para reproducibilidad
set.seed(2023)

# Realizamos 100 pasadas
for (i in 1:100) {
  # Particionamos los datos
  t.ids <- createDataPartition(df_wineSVM$quality_good, p = 0.7, list = FALSE)

  # Creación del modelo de red neuronal
  modelo <- neuralnet(
    formula,
    data = df_wineSVM[t.ids, ],
    hidden = c(5, 2), # Número de neuronas en las capas ocultas
    linear.output = TRUE # Utiliza salida lineal
  )

  # Realizamos la predicción con la red neuronal
  prediccion <- predict(modelo, newdata = df_wineSVM[-t.ids, ], type = "response")
  prediccion_binaria <- ifelse(prediccion > 1, 0, 1)

  # Creamos una matriz de confusión
  matriz_confusion_nn <- table(
    Real = df_wineSVM$quality_good[-t.ids][1:length(prediccion_binaria)],
    Predicho = prediccion_binaria
  )

  # Calculamos la precisión y sensibilidad con la red neuronal
  precision_nn <- (matriz_confusion_nn["0", "0"] + matriz_confusion_nn["1", "1"]) / sum(matriz_confusion_nn)
  sensibilidad_nn <- matriz_confusion_nn["1", "1"] / sum(matriz_confusion_nn["1", ])

  # Almacenamos los resultados en los vectores
  precision_vector_nn[i] <- precision_nn
  sensibilidad_vector_nn[i] <- sensibilidad_nn
}

# Calculamos el promedio de la precisión y sensibilidad con la red neuronal
promedio_precision_nn <- mean(precision_vector_nn, na.rm = TRUE)
promedio_sensibilidad_nn <- mean(sensibilidad_vector_nn, na.rm = TRUE)

# Imprimimos los resultados con la red neuronal
print(paste("Promedio de Precisión con Red Neuronal:", promedio_precision_nn))
print(paste("Promedio de Sensibilidad con Red Neuronal:", promedio_sensibilidad_nn))


[1] "Promedio de Precisión con Red Neuronal: 0.867265135699374"
[1] "Promedio de Sensibilidad con Red Neuronal: 1"


**Los Resultados**
La precision del SVM es de 99%, la de Randon forest es de casi 100% y en cambio la precision de Red Neuronal es de 86%.

**Conclusiones**
SVM está mostrando un rendimiento excepcionalmente alto en términos de precisión, pero habria que evaluar si es por un sobreajuste.

Random Forest al igual que con SVM, la precisión cercana al 100% en Random Forest sugiere un rendimiento excepcional. Sin embargo, al igual que antes, verifica si hay sobreajuste.

Red Neuronal aunque esta precisión de la red neuronal es ligeramente inferior, es importante destacar que podría estar generalizando mejor a nuevos datos y poder aprender patrones mas complejos.